In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers.experimental import preprocessing

Load Dataset

In [2]:
df = pd.read_csv('magic_gamma_telescope04_.csv')

In [3]:
df.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [4]:
df['class'].nunique()

2

In [5]:
df1 = pd.get_dummies(df['class'],drop_first = 'True')

In [6]:
df1.head()

,h
0,0
1,0
2,0
3,0
4,0


In [7]:
df['class']=df1

In [8]:
df.head()

,flength,fwidth,fsize,fconc,fconc1,fsym,fm3long,fm3trans,falpha,dist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,0
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,0
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,0
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,0
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,0


In [9]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.25)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

11412 train examples
3804 validation examples
3804 test examples


In [10]:
df1 = train.copy()

ds = tf.data.Dataset.from_tensor_slices((dict(df1))) #, df1.pop('class')))
for item in ds:
  print(item)

Streaming output truncated to the last 5000 lines.
{'flength': <tf.Tensor: shape=(), dtype=float64, numpy=60.5854>, 'fwidth': <tf.Tensor: shape=(), dtype=float64, numpy=23.5482>, 'fsize': <tf.Tensor: shape=(), dtype=float64, numpy=3.3166>, 'fconc': <tf.Tensor: shape=(), dtype=float64, numpy=0.2624>, 'fconc1': <tf.Tensor: shape=(), dtype=float64, numpy=0.1324>, 'fsym': <tf.Tensor: shape=(), dtype=float64, numpy=-45.4663>, 'fm3long': <tf.Tensor: shape=(), dtype=float64, numpy=-10.4143>, 'fm3trans': <tf.Tensor: shape=(), dtype=float64, numpy=-7.5557>, 'falpha': <tf.Tensor: shape=(), dtype=float64, numpy=1.557>, 'dist': <tf.Tensor: shape=(), dtype=float64, numpy=282.594>, 'class': <tf.Tensor: shape=(), dtype=uint8, numpy=0>}
{'flength': <tf.Tensor: shape=(), dtype=float64, numpy=220.603>, 'fwidth': <tf.Tensor: shape=(), dtype=float64, numpy=69.4505>, 'fsize': <tf.Tensor: shape=(), dtype=float64, numpy=3.7278>, 'fconc': <tf.Tensor: shape=(), dtype=float64, numpy=0.163>, 'fconc1': <tf.Tensor

In [11]:
def df_to_dataset(df1, shuffle=True, batch_size=3):
  '''Creates a tf.data dataset from a Pandas Dataframe'''
  df1 = df1.copy()
  labels = df1 #.pop('class')
  ds = tf.data.Dataset.from_tensor_slices((dict(df1), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df1))
  #We'll also want to process this a batch-size at a time, so we can use the batch method.  
  ds = ds.batch(batch_size)
  #Prefetching allows us to optimise fetching of data and training
  ds = ds.prefetch(batch_size)
  return ds

In [12]:
train_ds = df_to_dataset(train)
train_ds

<PrefetchDataset element_spec=({'flength': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fwidth': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fsize': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fconc': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fconc1': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fsym': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fm3long': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'fm3trans': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'falpha': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'dist': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'class': TensorSpec(shape=(None,), dtype=tf.uint8, name=None)}, TensorSpec(shape=(None, 11), dtype=tf.float64, name=None))>

In [13]:
[(train_features, label_batch)] = train_ds.take(1)
label_batch

<tf.Tensor: shape=(3, 11), dtype=float64, numpy=
array([[ 7.376890e+01,  1.067680e+01,  3.131700e+00,  2.182000e-01,
         1.158000e-01,  3.972410e+01,  5.623100e+01,  1.176940e+01,
         2.527350e+01,  1.702389e+02,  1.000000e+00],
       [ 9.232870e+01,  3.399420e+01,  3.847300e+00,  1.494000e-01,
         7.820000e-02, -5.456740e+01,  6.748720e+01,  1.869720e+01,
         1.521600e+00,  3.293570e+02,  0.000000e+00],
       [ 6.155530e+01,  2.372520e+01,  3.264900e+00,  1.798000e-01,
         8.990000e-02,  8.335120e+01,  3.941290e+01, -1.219380e+01,
         3.363000e+00,  2.140550e+02,  0.000000e+00]])>

In [14]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of flength:', train_features['flength'])
print('A batch of labels:', label_batch )

Every feature: ['flength', 'fwidth', 'fsize', 'fconc', 'fconc1', 'fsym', 'fm3long', 'fm3trans', 'falpha', 'dist', 'class']
A batch of flength: tf.Tensor([76.5111 36.8508 49.8322], shape=(3,), dtype=float64)
A batch of labels: tf.Tensor(
[[ 7.651110e+01  1.499670e+01  2.845400e+00  2.827000e-01  1.792000e-01
  -8.822380e+01  5.955040e+01 -1.122680e+01  7.478000e+00  2.745820e+02
   0.000000e+00]
 [ 3.685080e+01  1.989030e+01  2.769700e+00  3.195000e-01  1.878000e-01
   2.436590e+01  2.870910e+01 -1.478950e+01  6.038970e+01  8.007570e+01
   0.000000e+00]
 [ 4.983220e+01  9.084700e+00  2.894700e+00  3.927000e-01  2.105000e-01
   2.640390e+01  3.709350e+01  3.411100e+00  2.229160e+01  2.706941e+02
   1.000000e+00]], shape=(3, 11), dtype=float64)


In [15]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization(axis=None)

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # A pre-processing / non-traininable layer. 
  normalizer.adapt(feature_ds)

  return normalizer

In [16]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_tokens=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply one-hot encoding to our indices and return this feature
  return lambda feature: encoder(index(feature))

In [17]:
batch_size = 64
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [18]:
all_inputs = []
encoded_features = []

# Numeric features.
numeric_columns = ['flength','fwidth','fsize','fconc','fconc1','fsym','fm3long','fm3trans','falpha','dist','class']
for column_name in numeric_columns:
  numeric_column = tf.keras.Input(shape=(1,), name=column_name)
  normalization_layer = get_normalization_layer(column_name, train_ds)
  encoded_numeric_column = normalization_layer(numeric_column)
  all_inputs.append(numeric_column)
  encoded_features.append(encoded_numeric_column)

# Categorical features encoded as string.
#categorical_columns = ['class']
#for column_name in categorical_columns:
  #categorical_column = tf.keras.Input(shape=(1,), name=column_name, dtype='string')
  #encoding_layer = get_category_encoding_layer(column_name, train_ds, dtype='string', max_tokens=5)
  #encoded_categorical_column = encoding_layer(categorical_column)
  #all_inputs.append(categorical_column)
  #encoded_features.append(encoded_categorical_column)

In [19]:
all_inputs

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'flength')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fwidth')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fsize')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fconc')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fconc1')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fsym')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fm3long')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'fm3trans')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'falpha')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dist')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'class')>]

In [20]:
encoded_features

[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_1')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_2')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_3')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_4')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_5')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_6')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_7')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_8')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_9')>,
 <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'normalization_10')>]

In [21]:
#compile
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(128, activation="relu")(all_features)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.1)(x)
output = tf.keras.layers.Dense(1)(x)

#Optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum)
#model.compile(metrics=['accuracy'])
#model.fit(xtrain,ytrain,epochs=epochs,batch_size=batch_size)
#test_loss, test_acc = model.evaluate(xtest,ytest,verbose=2)
#print("Test Accuracy:", test_acc)

model = tf.keras.Model(all_inputs, output)

model.compile(optimizer='Adam', loss = tf.keras.losses.MeanSquaredError,
              metrics=["accuracy"])

In [22]:
model.fit(train_ds, validation_data=val_ds, epochs=1)

TypeError: ignored